# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [8]:
from azureml.core import Workspace, Experiment

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset

from sklearn.model_selection import train_test_split

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import choice

from azureml.core import Model
from azureml.core.webservice import AciWebservice, Webservice

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="hd_capstone")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-137812
Azure region: southcentralus
Subscription id: b968fb36-f06a-4c76-a15f-afab68ae7667
Resource group: aml-quickstarts-137812


In [3]:
cpu_cluster_name = "udacity-project3"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
dataset_link = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
dataset = Dataset.Tabular.from_delimited_files(path=dataset_link, header=False).to_pandas_dataframe()
dataset.columns =['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'iris_class'] 

dataset.head()

,sepal_length,sepal_width,petal_length,petal_width,iris_class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [5]:
train, test = train_test_split(dataset, test_size=0.2, random_state=42)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [9]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(
    evaluation_interval=3,
    slack_factor=0.1
)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling(
   parameter_space={
       "--C": uniform(0.1, 0.9),
       "--max_iter": choice(10, 50, 100)
    }
)


#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(
    source_directory=".",
    entry_script="train.py",
    compute_target=cpu_cluster,
)

hyperdrive_run_config = HyperDriveConfig(
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=100,
    max_concurrent_runs=5,
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    estimator=estimator
)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [11]:
#TODO: Submit your experiment
hd_run = exp.submit(hyperdrive_run_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [13]:
RunDetails(hd_run).show()
hd_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_cbed9cab-98c8-429b-be3c-4b3da9dcd4eb
Web View: https://ml.azure.com/experiments/hd_capstone/runs/HD_cbed9cab-98c8-429b-be3c-4b3da9dcd4eb?wsid=/subscriptions/b968fb36-f06a-4c76-a15f-afab68ae7667/resourcegroups/aml-quickstarts-137812/workspaces/quick-starts-ws-137812

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-02-07T13:27:18.433109][API][INFO]Experiment created<END>\n"<START>[2021-02-07T13:27:19.4158051Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2021-02-07T13:27:21.008303][GENERATOR][INFO]Trying to sample '5' jobs from the hyperparameter space<END>\n""<START>[2021-02-07T13:27:21.207135][GENERATOR][INFO]Successfully sampled '5' jobs, they will soon be submitted to the execution target.<END>\n"


## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run, best_model = hd_run.get_output()
best_run_metrics = best_run.get_metrics()

best_run

In [ ]:
best_model

In [ ]:
print('Best Run Id: ', best_run.id)
print('Accuracy:', best_run_metrics['accuracy'])
print(best_model._final_estimator)

In [ ]:
#TODO: Save the best model
#TODO: Save the best model
os.makedirs('./outputs', exist_ok=True)

joblib.dump(fitted_model, filename='outputs/hdmodel.joblib')

model_name = best_run.properties['model_name']
print(model_name)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
model = best_run.register_model(model_name = 'hd_model', 
                                                model_path='outputs/hdmodel.joblib')

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=2,
                                                enable_app_insights=True, auth_enabled=True)  

service_name = 'service_aml'
service = Model.deploy(ws, service_name, [model], deployment_config=aci_config)
service.wait_for_deployment(show_output = True)
print(service.state)
print(service.scoring_uri)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
test_ws = train.tail()
target = test_ws['iris_class']
test_ws = test_ws[:-1]

test_json = json.dumps({'data':test_ws.to_dict(orient='records')})
print(test_json)

results = service.run(teste_json)
print('Model results:', results)
print('Actual results:', y_actual.values)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())

service.delete()